In [ ]:
import numpy as np
template_traces = np.load('../data/traces_hawk1024_10000.npy')
template_x = np.load('../data/x_hawk1024_10000.npy')
len(template_traces)

In [ ]:
attack_traces =  np.load('../data/traces_hawk1024_new.npy')
attack_x =  np.load('../data/x_hawk1024_new.npy')
len(attack_traces)

In [ ]:
even_traces = []
even_x = []
even_attack_traces = []
even_attack_x = []

for i in range(10000):
    if template_x[i] % 2 == 0:
        even_traces.append(template_traces[i])
        even_x.append(template_x[i])

for i in range(1000):
    if attack_x[i] % 2 == 0:
        even_attack_traces.append(attack_traces[i])
        even_attack_x.append(attack_x[i])

In [ ]:

# code taken from http://wiki.newae.com/Tutorial_B8_Profiling_Attacks_(Manual_Template_Attack)

from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

def cov(x, y):
    return np.cov(x, y)[0][1]

def build_template(traces, x, hw, numPOIs, POIspacing):
    hw_max = len(hw)
    tempTraces = traces
    val = x

    # 2.5: Sort traces by HW
    tempTracesHW = [[] for _ in range(hw_max)]

    # Fill them up
    for i in range(len(tempTraces)):
        HW = hw[val[i]]
        tempTracesHW[HW].append(tempTraces[i])
    
    tempTracesHW = [np.array(tempTracesHW[HW]) for HW in range(hw_max)]

    # 3: Find averages
    tempMeans = np.zeros((hw_max, len(tempTraces[0])))
    for i in range(hw_max):
        tempMeans[i] = np.average(tempTracesHW[i], 0)

    # 4: Find sum of differences
    tempSumDiff = np.zeros(len(tempTraces[0]))
    for i in range(hw_max):
        for j in range(i):
            tempSumDiff += np.abs(tempMeans[i] - tempMeans[j])

    # 5: Find POIs
    POIs = []
    for i in range(numPOIs):
        nextPOI = tempSumDiff.argmax()
        POIs.append(nextPOI)
    
        poiMin = max(0, nextPOI - POIspacing)
        poiMax = min(nextPOI + POIspacing, len(tempSumDiff))
        for j in range(poiMin, poiMax):
            tempSumDiff[j] = 0

    # 6: Fill up mean and covariance matrix for each HW
    meanMatrix = np.zeros((hw_max, numPOIs))
    covMatrix  = np.zeros((hw_max, numPOIs, numPOIs))
    for HW in range(hw_max):
        for i in range(numPOIs):
            meanMatrix[HW][i] = tempMeans[HW][POIs[i]]
            for j in range(numPOIs):
                xi = tempTracesHW[HW][:,POIs[i]]
                yi = tempTracesHW[HW][:,POIs[j]]
                covMatrix[HW,i,j] = cov(xi, yi)
                
    return (meanMatrix, covMatrix, POIs)


In [ ]:
def attack(template, traces, x, hw):
    meanMatrix, covMatrix, POIs = template
    hw_max = len(hw)
    val = x

    success = 0
    failures = 0
    for j in range(len(traces)):
        P_k = np.zeros(hw_max)
        a = [traces[j][POIs[i]] for i in range(len(POIs))]
    
        for w in hw.values():
            rv = multivariate_normal(meanMatrix[w], covMatrix[w])
            p_kj = rv.pdf(a)

            P_k[w] = np.log(p_kj)

        guess = P_k.argsort()[-1]
        guess = list(hw.keys())[list(hw.values()).index(guess)]

        if guess == val[j]:
            success +=1
        else:
            failures += 1
    print(success, failures)

In [ ]:
start = 0
end = 1500
nbPOIs = 30
POIspacing = 10

# all values
model = {key: value for key, value in zip([i if i < 10 else 256 - i + 9 for i in range(16)], [_ for _ in range(16)])}
template = build_template([t[start:end] for t in template_traces], template_x, model, nbPOIs, POIspacing)
attack(template, [t[start:end] for t in attack_traces], attack_x, model)

# even values only
# model = {key: value for key, value in zip([2*i if i < 5 else 256 - 2*i + 8 for i in range(8)], [_ for _ in range(8)])}
# template = build_template([t[start:end] for t in even_traces], even_x, model, nbPOIs, POIspacing)
# attack(template, [t[start:end] for t in even_attack_traces], even_attack_x, model)